In [4]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import NearestNeighbors
import joblib
import pickle

In [ ]:
# Load dataset (Assuming it's a CSV file from Kaggle)
df = pd.read_csv('resume_dataset.csv')  # Replace with actual file name

In [8]:
# Load model
model = joblib.load(open('resume_screening_model.pkl', 'rb'))
vectorizer = joblib.load(open('tfidf_vectorizer.pkl', 'rb'))

In [10]:
# Advanced text preprocessing using spaCy
def nlp_preprocess(text):
    if isinstance(text, str):
        doc = nlp(text.lower())  # Convert to lowercase and process text
        tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
        return " ".join(tokens)
    return ""


<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
/var/folders/95/qrh427z549vgggs2kt3jz9r00000gn/T/ipykernel_43080/3763753363.py:6: SyntaxWarning: invalid escape sequence '\d'
  text = re.sub("\d+", "", text)  # Remove numbers


In [ ]:
# Load applicants' resumes (Assuming it's a CSV file with 'resume_text' column)
df_applicants = pd.read_csv('resumes.csv')  # Replace with actual file name
df_applicants = df_applicants[:5]
df_applicants['cleaned_resume'] = df_applicants['resume_text'].apply(clean_text)

In [ ]:
# Combine job title and job description for ranking
def combine_title_description(job_title, job_description):
    return clean_text(job_title) + " " + clean_text(job_description)

In [ ]:
# Resume ranking based on job title and description
def rank_resumes(job_title, job_description, top_n=5):
    job_text_combined = combine_title_description(job_title, job_description)
    job_vector = vectorizer.transform([job_text_combined])
    
    # Predict job category using the trained model
    predicted_category = model.predict(job_vector)[0]
    
    # Filter applicants' resumes by predicted category (if category exists in training data)
    filtered_df = df_applicants.copy()
    
    # Use Nearest Neighbors to find similar resumes among applicants
    if not filtered_df.empty:
        X_filtered = vectorizer.transform(filtered_df['cleaned_resume'])
        nn = NearestNeighbors(n_neighbors=min(top_n, len(filtered_df)), metric='cosine')
        nn.fit(X_filtered)
        distances, indices = nn.kneighbors(job_vector)
        return filtered_df.iloc[indices[0]]
    else:
        return pd.DataFrame()  # Return empty DataFrame if no resumes are found

In [ ]:
# Example usage
job_title = "Designer"
job_desc = "Looking for a designer to create visually compelling designs for digital and print media, ensuring alignment with brand guidelines and collaborating with teams to deliver creative solutions. Proficiency in design tools and a strong portfolio are required."
top_resumes = rank_resumes(job_title, job_desc, top_n=5)
print(top_resumes[['resume_text']])

In [122]:
#final step above 